In [1]:
def txt_to_list(path):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
        text_list = list()
        for i in text:
            text_list.append(i)
        label = text_label[2:]
        label_list = ['O' for i in range(len(text_list))]
        for i in label:
            entity = i.split('\t')
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        x.append(text_list)
        y.append(label_list)
    return [x, y]

In [2]:
x1, y1 = txt_to_list('../datasets/stage1/SampleData_deid.txt')

In [3]:
x2, y2 = txt_to_list('../datasets/stage2/train_1_update.txt')

In [4]:
x4, y4 = txt_to_list('../datasets/stage4/train_2.txt')

In [5]:
x = x1 + x2 + x4
len(x)

346

In [6]:
y = y1 + y2 + y4
len(y)

346

In [7]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(valid_x), len(valid_y))
print(len(test_x), len(test_y))

220 220
56 56
70 70


In [14]:
import kashgari
kashgari.config.use_cudnn_cell = True
from kashgari.tasks.labeling.models import BiLSTM_Model

model = BiLSTM_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10)
model.evaluate(test_x, test_y)
model.save('model/BiLSTM')

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 4211)]            0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 4211, 100)         172400    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 4211, 256)         235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 4211, 256)         0         
_________________________________________________________________
layer_time_distributed (Time (None, 4211, 28)          7196      
_________________________________________________________________
activation_2 (Activation)    (None, 4211, 28)          0         
Total params: 415,116
Trainable params: 415,116
Non-trainable params: 0
_____________________________________________________

In [19]:
# loaded_model = kashgari.utils.load_model('model/BiLSTM')
# loaded_model.predict(test_x)[0]

In [16]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_Model
from kashgari.embeddings import BERTEmbedding

bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING)
model = BiLSTM_Model(bert_embed)
model = BiLSTM_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10)
model.evaluate(test_x, test_y)
model.save('model/BERT_BiLSTM')

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 4211)]            0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 4211, 100)         172400    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 4211, 256)         235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 4211, 256)         0         
_________________________________________________________________
layer_time_distributed (Time (None, 4211, 28)          7196      
_________________________________________________________________
activation_3 (Activation)    (None, 4211, 28)          0         
Total params: 415,116
Trainable params: 415,116
Non-trainable params: 0
_____________________________________________________

In [20]:
# loaded_model = kashgari.utils.load_model('model/BERT_BiLSTM')
# loaded_model.predict(test_x)[0]